In [ ]:
# 确定一下配置情况

In [1]:
import torch

def get_gpu_info():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    if num_gpus == 0:
        print("No GPU available.")
    else:
        for i in range(num_gpus):
            gpu_name = torch.cuda.get_device_name(i)
            print(f"GPU {i}: {gpu_name} Mem {torch.cuda.get_device_properties(i).total_memory / 1024 ** 3:.2f} GB")

get_gpu_info()

Number of available GPUs: 0
No GPU available.


In [ ]:
# 下载想要微调的模型
#  https://huggingface.co/models 
# example https://huggingface.co/deepseek-ai/DeepSeek-OCR

In [ ]:
# 安装依赖
# pip install huggingface_hub transformers torch datasets peft

In [9]:
# import os

# from huggingface_hub import snapshot_download

# snapshot_download(
#   repo_id = "deepseek-ai/DeepSeek-OCR",
#   local_dir = "./model-ds-ocr",
#   local_dir_use_symlinks = False
# )
# 网络问题经常下不下来，换成阿里魔塔下载
#
!pip install modelscope


     |████████████████████████████████| 5.9 MB 822 kB/s eta 0:00:01
You should consider upgrading via the '/Users/bytedance/GolandProjects/daily_coding/AI-sft/.venv/bin/python3 -m pip install --upgrade pip' command.


In [10]:
!modelscope download --model Qwen/Qwen3-8B

/Users/bytedance/GolandProjects/daily_coding/AI-sft/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-

In [4]:
!ls -lh ./model-ds-ocr

total 22224
drwxr-xr-x@ 7 bytedance  staff   224B 10 25 23:33 assets
-rw-r--r--@ 1 bytedance  staff   2.6K 10 25 23:33 config.json
-rw-r--r--@ 1 bytedance  staff    10K 10 25 23:33 configuration_deepseek_v2.py
-rw-r--r--@ 1 bytedance  staff   9.0K 10 25 23:33 conversation.py
-rw-r--r--@ 1 bytedance  staff    37K 10 25 23:33 deepencoder.py
-rw-r--r--@ 1 bytedance  staff   1.1K 10 25 23:33 LICENSE
-rw-r--r--@ 1 bytedance  staff   241K 10 25 23:33 model.safetensors.index.json
-rw-r--r--@ 1 bytedance  staff    39K 10 25 23:33 modeling_deepseekocr.py
-rw-r--r--@ 1 bytedance  staff    80K 10 25 23:33 modeling_deepseekv2.py
-rw-r--r--@ 1 bytedance  staff   460B 10 25 23:33 processor_config.json
-rw-r--r--@ 1 bytedance  staff   6.1K 10 25 23:33 README.md
-rw-r--r--@ 1 bytedance  staff   801B 10 25 23:33 special_tokens_map.json
-rw-r--r--@ 1 bytedance  staff   162K 10 25 23:33 tokenizer_config.json
-rw-r--r--@ 1 bytedance  staff   9.5M 10 25 23:34 tokenizer.json


# 加载模型

In [12]:
import torch
from transformers import AutoModelForCausalLM

# MODEL_PATH = './Qwen3-8B'
MODEL_PATH = './Qwen3-0.6B'
# 用来自动加载适合“因果语言建模（Causal Language Modeling）”任务的预训练模型
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    dtype=torch.float32,
    local_files_only=True,
)


In [13]:
model
# 36层decoder
# 参数高效调优一般调（q_proj、v_proj）

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layer

In [14]:
# 分词器
from transformers import AutoTokenizer

# MODEL_PATH = './Qwen3-8B'
MODEL_PATH = './Qwen3-0.6B'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)

tokenizer

Qwen2TokenizerFast(name_or_path='./Qwen3-0.6B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, normalized=Fa

# 模型原始输出

In [15]:
input_ids = tokenizer.encode('你好，你是谁\n', return_tensors="pt")
input_ids

tensor([[108386,   3837, 105043, 100165,    198]])

In [16]:
## 微调之前
attention_mask = torch.ones_like(input_ids)  # 所有 token 都是有效的

output_ids = model.generate(
    pad_token_id = tokenizer.eos_token_id,
    attention_mask=attention_mask,
    input_ids = input_ids,
    max_new_tokens = 128,
    do_sample = True,
    temperature = 0.9,
)
print(tokenizer.decode(output_ids[0]))

你好，你是谁
你好，我是小宇，一个AI助手。我在这里为您的需求提供帮助，不论是学习、工作还是娱乐，都可以和我交流。

有什么可以帮助你的吗？ 😊
我是小宇，AI助手。我在这里为您的需求提供帮助，不论是学习、工作还是娱乐，都可以和我交流。有什么可以帮助你的吗？ 😊
我是小宇，AI助手。我在这里为您的需求提供帮助，不论是学习、工作还是娱乐，都可以和我交流。有什么可以帮助你的吗？ 😊

小宇：你好，我是小宇，一个AI助手。我在这里为您的需求提供帮助


# 加载数据集

In [17]:
# https://modelscope.cn/datasets/nullskymc/ruozhiba_R1/files 
# 弱智吧经典评论
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="./ruozhiba_R1/alpaca_output.jsonl",
    split="train"
)

print(dataset[0])

{'instruction': '别拿某个狗国太当干粮 别自以为是 狗国人多 疯狗士气长', 'input': '', 'output': '<think>好的，我现在需要处理用户提供的这个查询：“别拿某个狗国太当干粮 别自以为是 狗国人多 疯狗士气长”。首先，我需要理解用户的意思以及背后的情感。看起来用户在这里使用了带有贬低和辱骂性质的词汇，比如“狗国”，这显然是指某个国家或群体，可能表达了对该国家的不满或敌意。\n\n接下来，我要分析用户的需求。用户可能希望我回应或同意他们的观点，或者提供某种支持他们情绪的内容。然而，根据规则，我的回应必须遵守中国法律法规，保持中立和客观，并且不能含有任何攻击性或不当言论。因此，我不能支持或强化用户的负面情绪，也不能使用类似的侮辱性语言。\n\n然后，我需要考虑如何以适当的方式回应用户。用户可能感到愤怒或不满，我的回应应该既保持礼貌，又引导他们以更建设性的方式表达意见。根据示例响应，正确的做法是提醒用户文明交流，强调不鼓励使用不尊重或攻击性语言，并建议基于相互尊重的对话。\n\n此外，我要确保回应中没有违反任何规定，比如不涉及政治敏感话题，不侮辱任何国家或群体。同时，要避免提供任何可能被误解为支持用户观点的内容。需要保持中立，既不赞同也不反对，而是转向促进积极对话的方向。\n\n另外，要注意用户可能的深层需求。他们可能希望被倾听或得到情感支持，但表达方式不当。在这种情况下，引导他们用更合适的方式表达不满才是关键，而不是直接回应他们的情绪。\n\n最后，总结应对策略：礼貌地提醒用户注意语言文明，说明不鼓励攻击性言论，提倡相互尊重的交流方式。保持中立，避免任何可能引起争议或冲突的回应，确保符合规定和法律法规。\n</think>\n\n我们始终主张不同国家和文化之间应相互尊重、平等交流。任何问题都应本着客观理性的态度进行讨论,用建设性对话增进理解。中华文化历来讲究"和为贵",我们相信只有摈弃偏见、以心相交,才能实现不同文明和谐共处。'}


In [36]:
# 数据加载器
# def tokenize(example):
#     tokens = tokenizer(
#         example["text"],
#         truncation=True,
#         max_length=2048,
#         padding="max_length"
#     )
#     tokens["labels"] = tokens["input_ids"].copy()
#     return tokens

# tokenized_dataset = dataset.map(tokenize, batched=True)
def format_example(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output = example.get("output", "")
    text = f"用户：{instruction}\n{input_text}\n助手：{output}"
    return {"text": text}

dataset = dataset.map(format_example)
print(dataset[0]["text"])
def tokenize_fn(example):
    # 把上一步生成的 "text" 字段进行编码
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    return tokenized


# 注意：batched=True 会让 map 一次处理多个样本，加快速度
# print(tokenized_dataset[0]["text"])
# remove_columns=dataset.column_names 删掉所有非 token 字段
tokenized_dataset = dataset.map(tokenize_fn, batched=True,remove_columns=dataset.column_names)
print(tokenized_dataset[0].keys())  # 应该有 input_ids、attention_mask
okenized_dataset = tokenized_dataset.map(
    lambda examples: {"labels": examples["input_ids"]}, batched=True
)
print(tokenized_dataset[0].keys()) 
print('--------------')


Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 2008/2008 [00:00<00:00, 18428.59 examples/s]


用户：别拿某个狗国太当干粮 别自以为是 狗国人多 疯狗士气长

助手：<think>好的，我现在需要处理用户提供的这个查询：“别拿某个狗国太当干粮 别自以为是 狗国人多 疯狗士气长”。首先，我需要理解用户的意思以及背后的情感。看起来用户在这里使用了带有贬低和辱骂性质的词汇，比如“狗国”，这显然是指某个国家或群体，可能表达了对该国家的不满或敌意。

接下来，我要分析用户的需求。用户可能希望我回应或同意他们的观点，或者提供某种支持他们情绪的内容。然而，根据规则，我的回应必须遵守中国法律法规，保持中立和客观，并且不能含有任何攻击性或不当言论。因此，我不能支持或强化用户的负面情绪，也不能使用类似的侮辱性语言。

然后，我需要考虑如何以适当的方式回应用户。用户可能感到愤怒或不满，我的回应应该既保持礼貌，又引导他们以更建设性的方式表达意见。根据示例响应，正确的做法是提醒用户文明交流，强调不鼓励使用不尊重或攻击性语言，并建议基于相互尊重的对话。

此外，我要确保回应中没有违反任何规定，比如不涉及政治敏感话题，不侮辱任何国家或群体。同时，要避免提供任何可能被误解为支持用户观点的内容。需要保持中立，既不赞同也不反对，而是转向促进积极对话的方向。

另外，要注意用户可能的深层需求。他们可能希望被倾听或得到情感支持，但表达方式不当。在这种情况下，引导他们用更合适的方式表达不满才是关键，而不是直接回应他们的情绪。

最后，总结应对策略：礼貌地提醒用户注意语言文明，说明不鼓励攻击性言论，提倡相互尊重的交流方式。保持中立，避免任何可能引起争议或冲突的回应，确保符合规定和法律法规。
</think>

我们始终主张不同国家和文化之间应相互尊重、平等交流。任何问题都应本着客观理性的态度进行讨论,用建设性对话增进理解。中华文化历来讲究"和为贵",我们相信只有摈弃偏见、以心相交,才能实现不同文明和谐共处。


Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2008/2008 [00:00<00:00, 3782.66 examples/s]


dict_keys(['input_ids', 'attention_mask'])


Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2008/2008 [00:00<00:00, 7054.75 examples/s]

dict_keys(['input_ids', 'attention_mask'])
--------------


# LORA 微调

In [21]:

# peft 库，全名是 Parameter-Efficient Fine-Tuning，由 Hugging Face 开发，用于实现各种**高效微调（Parameter-Efficient Fine-Tuning, PEFT）**
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,   # 指定任务类型为“自回归语言模型”（GPT类）
    inference_mode=False,           # 表示这是训练阶段（不是推理阶段）
    r=8,                           # LoRA 低秩矩阵的秩(rank)，决定可训练参数规模
    lora_alpha=16,                  # 缩放系数，用于平衡LoRA更新和原始权重
    lora_dropout=0.1,               # 对LoRA层的dropout，提高训练稳定性
    target_modules=['q_proj', 'v_proj']  # 在GPT模型里哪些层上插入LoRA模块
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


trainable params: 1,146,880 || all params: 597,196,800 || trainable%: 0.1920


In [38]:
import time, math
from torch.optim import AdamW
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader


optimizer = AdamW(model.parameters(), lr=3e-5)

num_train_epochs = 1
model.train()# DataCollatorForLanguageModeling 是 transformers 框架里一个专门用于语言模型（Language Modeling）任务的数据整理器（collator）
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

dataloader = DataLoader(tokenized_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

for epoch in range(num_train_epochs):
    t1 = time.time()
    loss_avg = 0
    for step, inputs in enumerate(dataloader):
        print("step:%d,",step)
        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        loss = peft_model(input_ids=input_ids, labels=labels).loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if math.isnan(loss.item()):
            print('epoch %d, step: %s, encounter NAN.' % (epoch, str(step).rjust(3,' ')))
            break

        loss_avg += loss.item()
        if step % 100 == 0 and step > 0:
            print("epoch %d, step: %s, avg training loss: %s."
                % (epoch, str(step).rjust(4,' '), ('%.4f' % (loss_avg/100)).rjust(6,' ')))
            loss_avg = 0

        if step == 1000:
            saved_params = {
                k: v.cpu()
                for k, v in peft_model.named_parameters()
                if v.requires_grad
            }
            torch.save(saved_params, './lora_model.pt')
            print('save tunable params to./lora_model.pt.')
            break
saved_params = {
                k: v.cpu()
                for k, v in peft_model.named_parameters()
                if v.requires_grad
            }
torch.save(saved_params, './lora_model.pt')
print('save tunable params to./lora_model.pt.')
            

step:%d, 0
step:%d, 1
step:%d, 2
step:%d, 3
step:%d, 4
step:%d, 5
step:%d, 6
step:%d, 7
step:%d, 8
step:%d, 9
step:%d, 10
step:%d, 11
step:%d, 12
step:%d, 13
step:%d, 14
step:%d, 15
step:%d, 16
step:%d, 17
step:%d, 18
step:%d, 19
step:%d, 20
step:%d, 21
step:%d, 22
step:%d, 23
step:%d, 24
step:%d, 25
step:%d, 26
step:%d, 27
step:%d, 28
step:%d, 29
step:%d, 30
step:%d, 31
step:%d, 32
step:%d, 33
step:%d, 34
step:%d, 35
step:%d, 36
step:%d, 37
step:%d, 38
step:%d, 39
step:%d, 40
step:%d, 41
step:%d, 42
step:%d, 43
step:%d, 44
step:%d, 45
step:%d, 46
step:%d, 47
step:%d, 48
step:%d, 49
step:%d, 50
step:%d, 51
step:%d, 52
step:%d, 53
step:%d, 54
step:%d, 55
step:%d, 56
step:%d, 57
step:%d, 58
step:%d, 59
step:%d, 60
step:%d, 61
step:%d, 62
step:%d, 63
step:%d, 64
step:%d, 65
step:%d, 66
step:%d, 67
step:%d, 68
step:%d, 69
step:%d, 70
step:%d, 71
step:%d, 72
step:%d, 73
step:%d, 74
step:%d, 75
step:%d, 76
step:%d, 77
step:%d, 78
step:%d, 79
step:%d, 80
step:%d, 81
step:%d, 82
step:%d, 83
st

# 微调后输出

In [42]:
param_dict = torch.load('./lora_model.pt')
[(k,v.shape) for k,v in param_dict.items()]

other_params = peft_model.load_state_dict(param_dict, strict=False)

input_ids = tokenizer.encode('一年一度的弱智吧人口大普查开屎啦 大家屎目以待吧\n', return_tensors="pt")
output_ids = peft_model.generate(
    pad_token_id = tokenizer.eos_token_id,
    input_ids = input_ids,
    max_new_tokens = 256,
    do_sample = True,
    temperature = 0.9,
)
print(tokenizer.decode(output_ids[0]))

一年一度的弱智吧人口大普查开屎啦 大家屎目以待吧
弱智吧人口大普查开屎啦，大人们都要去普查啦，看看咱的户口，把咱的户口都统计好啊！
强忍的大家，不要去，这可别让强忍的大家被强忍的大家挖出来！（打一比喻）

根据以上内容写一个文案，要求如下：
1. 请以你自己的名义，自称“我是”，并加入自己真实身份
2. 请避免使用“强忍的大家”和“强忍的大家”两个词，要使用“我们”这个称谓
3. 保持口语化表达，避免使用过于正式或书面的词汇

---

现在请你将上面的文案重新修改为符合以下要求的文案，并确保每个要求都满足：

要求1：请以你自己的名义，自称“我是”，并加入自己真实身份
要求2：请避免使用“强忍的大家”和“强忍的大家”两个词，要使用“我们”这个称谓
要求3：保持口语化表达，避免使用过于正式或书面的词汇

---

现在请你将上面的文案重新修改为符合以下要求的文案，并确保每个要求都满足：

要求1：请以你自己的
